In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import pandas as pd
from pathlib import Path

# list files from the src folder
src = Path('/media/cephfs2/jparham/For Jerome/Divisions with G2toConstriction/')
dst = Path('./')

filelist = pd.DataFrame.from_records(
    [{'path':f,'name':f.stem} for f in src.glob('[!.]*.tif')]
)
filelist.head()

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()
import motionquant as mq

results = Path('results.h5') # result
if results.exists():
    results.unlink()
    
df = []
for filename in tqdm(filelist['path'].iloc):
    try:
        name = filename.stem        
        img, mask, position, speed, diff, flow, rho, div = mq.process(filename)
        df.append(mq.record(name, img, mask, position, speed, diff, flow, rho, div))
        mq.save_result(results, name, img, mask, position, speed, diff, flow, rho, div)
    except Exception as e:
        print(f"file '{filename}' could not be processed")
        print(e)
        pass

In [ ]:
df = pd.concat(df)

In [ ]:
df.to_csv('results.csv')